# PROYECTO FINAL

La empresa GoGreen Bikesharing, se dedica al alquiler de bicicletas y posee datos tales como la cantidad de bicis alquiladas por usuarios registrados, la cantidad de alquileres realizados por usuarios puntuales, y la cantidad total. 

A estos datos se les añadió información meteorológica, y el calendario de festivos.

Ahora buscan analizar cuáles son los aspectos que más influyen en la cantidad de bicis que van a alquilar en un día.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sidetable
import datetime

PREGUNTAS

¿en que momento del año se alquilan mas bicicletas?

Influye soleado o nubes?

Fin de semana o laborable?

los primeros y ultimos dias del mes hay mas reservas?

festivos aumenta las reservas?

la tempreatura a partir de la cual disminuye los alquileres  ->Punto ruptura

Punto maximo de alquiler

la temperatura alta influye al alquiler de espontaneos o habituales?

la humedad es un punto clave en algun momento? A partir de que punto de humedad bajan drasticamente las reservas?

fecha donde hay mas y menos reservas






In [2]:
df_bike = pd.read_csv("./bikes.csv", index_col=0)
df_bike.sample(6)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
445,446,22-03-2019,summer,1,3,0,4,1,1,22.720847,26.57835,83.1250,7.876654,1334,5537,6871
109,110,20-04-2018,summer,0,4,0,3,1,1,24.395000,28.21960,61.4167,16.208975,613,3331,3944
322,323,19-11-2018,winter,0,11,0,6,0,1,13.495847,16.22415,50.2083,15.041232,943,2720,3663
68,69,10-03-2018,spring,0,3,0,4,1,3,15.952731,19.28340,0.0000,17.545759,46,577,623
473,474,19-04-2019,summer,1,4,0,4,1,1,20.431653,24.65230,61.2500,4.417256,846,5719,6565
702,703,04-12-2019,winter,1,12,0,2,1,1,19.509153,23.45270,73.3750,11.666643,551,6055,6606


In [3]:
df_bike.mnth.dtype

dtype('int64')

# limpieza string

In [4]:
df_bike.season.unique()

array(['spring', 'summer', 'autumn', 'winter'], dtype=object)

# duplicados

In [5]:
df_bike.duplicated().sum()

0

# Nulos

In [6]:
df_bike.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [8]:
df_bike.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    object 
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(10), object(2)
memory usage: 97.0+ KB


# separando la col DTime en day, month, year

In [9]:
df_bike.head(2)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801


In [11]:
df_bike["day"] = df_bike["dteday"].str.split(pat= "-", n = -1, expand = True)[0]

In [14]:
df_bike.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,day
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,01
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,02
2,3,03-01-2018,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349,03
3,4,04-01-2018,spring,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562,04
4,5,05-01-2018,spring,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600,05


In [21]:
df_bike=df_bike.reindex(columns=['instant', 'dteday', 'season', 'yr', 'mnth','day', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'])

In [13]:
df_bike["day"].dtypes

dtype('O')

In [25]:
df_bike["day"] =df_bike["day"].astype("int64")

In [26]:
df_bike["day"].dtypes

dtype('int64')

In [27]:
df_bike.head()

,instant,dteday,season,yr,mnth,day,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,2,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,spring,0,1,3,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,spring,0,1,4,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,spring,0,1,5,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [41]:
df_bike.weekday.unique()

array([6, 0, 1, 2, 3, 4, 5])

In [45]:
df_bike.stb.counts().T

,workingday,yr,holiday,weathersit,season,weekday,mnth,day,temp,hum,casual,windspeed,registered,atemp,cnt,dteday,instant
count,730,730,730,730,730,730,730,730,730,730,730,730,730,730,730,730,730
unique,2,2,2,3,4,7,12,31,498,594,605,649,678,689,695,730,730
most_freq,1,0,0,1,autumn,6,1,1,26.035,61.3333,120,9.041918,4841,32.7344,5409,01-01-2018,1
most_freq_count,499,365,709,463,188,105,62,24,5,4,4,3,3,4,2,1,1
least_freq,0,0,1,3,winter,3,2,31,10.865,79.7083,2290,4.417256,3100,26.3571,5740,01-01-2018,1
least_freq_count,231,365,21,21,178,103,56,14,1,1,1,1,1,1,1,1,1


In [46]:
df_weekday= df_bike.groupby("weekday"), 
df_weekday

In [ ]:
df_weekday= df_bike.isin()

In [ ]:
frecuentes= df["Name"].value_counts()[0:50]
frecuentes

In [ ]:
values = frecuentes.keys().to_list()
values

In [ ]:
df_name = df[df["Name"].isin(values)]
df_name

# definir categoricas y numericas
# .reindex las columnas a gusto o uso
# .describe
# .stb.counts()
# .stb.missing()
# pairplot - Busca relaciones - ¿Hay que hacer algun ajuste?
# Boxplot
# Outliers - Identificarlos ¿Que hacer con ellos? %de cada columna
# Heatmap con la relacion
# ajuste lineal multiple
